In [17]:
# use this file to return data on receptions and targets
# there are multiple functions that return the data

In [18]:
# Import the necessary modules 
import nfl_data_py as nfl
import pandas as pd
from IPython.display import display, HTML

# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# FIXME - minor issue 
# Define the years of interest
years = list(range(2017, 2025)) # for the range you must select the following year to return the current year data

# Base columns for all positions
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name',
    'position', 'position_group', 'recent_team', 'opponent_team',
    'fantasy_points', 'fantasy_points_ppr'
]

# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]


# Define the WR-specific target-related columns
wr_target_columns = ['targets', 'target_share', 'receptions', 'receiving_yards']

# Combine base columns with WR-specific columns
wr_all_columns = base_columns + wr_columns
wr_tgt_columns = base_columns + wr_target_columns

# Import data for all the specified years
nfl_data_all_years = nfl.import_weekly_data(
    years=years,
    columns=wr_all_columns
)

# Filter to show only WR positions
wr_data_all_years = nfl_data_all_years[nfl_data_all_years['position'] == 'WR']

# Select only the relevant columns for WRs
wr_tgt_data_all_years = wr_data_all_years[wr_tgt_columns]

Downcasting floats.


In [19]:
# Function returns WR with at least 7 targets per game for each week with defined year range 
def wr_tgts_7plus_per_game():
    
    # Filter WRs with at least 7 targets per game for each week
    wr_with_min_7_targets_all_years = wr_tgt_data_all_years[wr_tgt_data_all_years['targets'] >= 7]
    
    # Sort by year, week, and then by targets for better analysis
    wr_with_min_7_targets_sorted_all_years = wr_with_min_7_targets_all_years.sort_values(by=['season', 'week', 'targets'], ascending=[False, True, False])
    
    # Remove the 'player_id' and 'position_group' columns
    wr_filtered_data_all_years = wr_with_min_7_targets_sorted_all_years.drop(columns=['player_id', 'position_group'])
    
    # Format decimals to two decimal places for all columns
    wr_filtered_data_all_years = wr_filtered_data_all_years.round(2)
    
    # Specifically format 'fantasy_points' and 'fantasy_points_ppr' as strings with two decimal places
    wr_filtered_data_all_years['fantasy_points'] = wr_filtered_data_all_years['fantasy_points'].apply(lambda x: f"{x:.2f}")
    wr_filtered_data_all_years['fantasy_points_ppr'] = wr_filtered_data_all_years['fantasy_points_ppr'].apply(lambda x: f"{x:.2f}")

    # Export the DataFrame to a CSV file
    wr_filtered_data_all_years.to_csv('wr_tgts_7plus_per_game.csv', index=False)
    
    # Display the refined data
    return display(HTML(wr_filtered_data_all_years.to_html(index=False)))

wr_tgts_7plus_per_game()

season,season_type,week,player_name,position,recent_team,opponent_team,fantasy_points,fantasy_points_ppr,targets,target_share,receptions,receiving_yards
2024,REG,1,C.Kupp,WR,LA,DET,18.00,32.00,21,0.44,14,110.0
2024,REG,1,T.Hill,WR,MIA,JAX,19.00,26.00,12,0.33,7,130.0
2024,REG,1,C.Sutton,WR,DEN,SEA,3.80,7.80,12,0.29,4,38.0
2024,REG,1,W.Robinson,WR,NYG,MIN,5.80,11.80,12,0.32,6,44.0
2024,REG,1,K.Allen,WR,CHI,TEN,2.90,6.90,11,0.38,4,29.0
2024,REG,1,G.Wilson,WR,NYJ,SF,6.00,12.00,11,0.38,6,60.0
2024,REG,1,A.Brown,WR,PHI,GB,17.90,22.90,10,0.34,5,119.0
2024,REG,1,C.Lamb,WR,DAL,CLE,8.60,13.60,10,0.32,5,61.0
2024,REG,1,Z.Flowers,WR,BAL,KC,5.10,11.10,10,0.25,6,37.0
2024,REG,1,A.Cooper,WR,CLE,DAL,1.60,3.60,9,0.21,2,16.0


In [20]:
# function returns the count for the number of times the targets>=7 for each season grouped by year and ordered by count
def wr_tgts_7plus_per_game_total_ordered_by_count():

    # Filter WRs with at least 7 targets per game for each week
    wr_with_min_7_targets_all_years = wr_tgt_data_all_years[wr_tgt_data_all_years['targets'] >= 7]
    
    # Group by year, player, and count the number of weeks
    wr_tgts_count_per_year = wr_with_min_7_targets_all_years.groupby(['season', 'player_name']).size().reset_index(name='count_of_weeks')
    
    # Sort the results by year and count of weeks in descending order
    wr_tgts_count_per_year_sorted = wr_tgts_count_per_year.sort_values(by=['season', 'count_of_weeks'], ascending=[False, False])

    # Export the DataFrame to a CSV file
    wr_tgts_count_per_year_sorted.to_csv('wr_tgts_7plus_per_game_total_ordered_by_count.csv', index=False)
    
    # Display the results
    return display(HTML(wr_tgts_count_per_year_sorted.to_html(index=False)))

wr_tgts_7plus_per_game_total_ordered_by_count()

season,player_name,count_of_weeks
2024,A.Cooper,5
2024,C.Lamb,5
2024,C.Godwin,4
2024,D.London,4
2024,D.Metcalf,4
2024,D.Moore,4
2024,G.Pickens,4
2024,G.Wilson,4
2024,J.Jefferson,4
2024,J.Smith-Njigba,4


In [21]:
# function reuturns the total count of games where the WR had 7+ targets for each season
def wr_tgts_7plus_total_per_year_by_player():

    # Filter WRs with at least 7 targets per game for each week
    wr_with_min_7_targets_all_years = wr_tgt_data_all_years[wr_tgt_data_all_years['targets'] >= 7]
    
    # Group by year, player, and count the number of weeks
    wr_tgts_count_per_year = wr_with_min_7_targets_all_years.groupby(['season', 'player_name']).size().reset_index(name='count_of_weeks')
    
    # Pivot the table to have players as rows and seasons as columns
    wr_tgts_pivot = wr_tgts_count_per_year.pivot_table(index='player_name', columns='season', values='count_of_weeks', fill_value=0)
    
    # Sort the columns (years) in descending order
    wr_tgts_pivot = wr_tgts_pivot[sorted(wr_tgts_pivot.columns, reverse=True)]
    
    # Sort the rows by the most recent season's count in descending order
    wr_tgts_pivot_sorted = wr_tgts_pivot.sort_values(by=wr_tgts_pivot.columns[0], ascending=False)

    # Export the DataFrame to a CSV file
    wr_tgts_pivot_sorted.to_csv('wr_tgts_7plus_total_per_year_by_player.csv')
    
    # Display the results
    return display(HTML(wr_tgts_pivot_sorted.to_html(index=True)))

# call the function
wr_tgts_7plus_total_per_year_by_player()

season,2024,2023,2022,2021,2020,2019,2018,2017
player_name,,,,,,,,
A.Cooper,5.0,11.0,10.0,8.0,10.0,10.0,11.0,6.0
C.Lamb,5.0,15.0,15.0,9.0,10.0,0.0,0.0,0.0
J.Smith-Njigba,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
D.Moore,4.0,11.0,7.0,17.0,9.0,11.0,8.0,0.0
J.Jefferson,4.0,8.0,14.0,15.0,9.0,0.0,0.0,0.0
M.Nabers,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M.Pittman,4.0,12.0,12.0,9.0,4.0,0.0,0.0,0.0
N.Collins,4.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0
G.Wilson,4.0,14.0,12.0,0.0,0.0,0.0,0.0,0.0


In [22]:
# function returns WR recptions >=30 and ordered by yards per reception for each year
def wr_rec_30plus_and_order_by_yds_per_rec():
    
    # Group by season and player, then sum the relevant statistics
    wr_grouped_data = wr_tgt_data_all_years.groupby(['season', 'player_name', 'recent_team'], as_index=False).agg({
        'receptions': 'sum',
        'receiving_yards': 'sum'
    })

    # Filter WRs with at least 30 receptions in a season and create a copy of the DataFrame to avoid the warning
    wr_filtered_by_receptions = wr_grouped_data[wr_grouped_data['receptions'] >= 30].copy()
    
    # Calculate yards per reception using .loc[] to avoid the SettingWithCopyWarning
    wr_filtered_by_receptions['yards_per_reception'] = wr_filtered_by_receptions['receiving_yards'] / wr_filtered_by_receptions['receptions']
    
    # Rank WRs by yards per reception in descending order
    wr_ranked_by_yards_per_reception = wr_filtered_by_receptions.sort_values(by=['season', 'yards_per_reception'], ascending=[False, False])
    
    # Format decimals to two decimal places
    wr_ranked_by_yards_per_reception = wr_ranked_by_yards_per_reception.round(2)

    # Export the DataFrame to a CSV file
    wr_ranked_by_yards_per_reception.to_csv('wr_rec_30plus_and_order_by_yds_per_rec.csv', index=False)
    
    # Display the ranked WRs
    return display(HTML(wr_ranked_by_yards_per_reception.to_html(index=False)))

# call the function
wr_rec_30plus_and_order_by_yds_per_rec()

season,player_name,recent_team,receptions,receiving_yards,yards_per_reception
2024,N.Collins,HOU,32,567.0,17.72
2024,D.Metcalf,SEA,31,469.0,15.13
2024,C.Godwin,TB,32,386.0,12.06
2024,D.London,ATL,32,354.0,11.06
2024,M.Nabers,NYG,35,386.0,11.03
2024,S.Diggs,HOU,31,315.0,10.16
2024,J.Smith-Njigba,SEA,34,310.0,9.12
2024,G.Wilson,NYJ,33,292.0,8.85
2024,W.Robinson,NYG,32,230.0,7.19
2023,B.Aiyuk,SF,84,1491.0,17.75
